In [1]:
import open3d as o3d
from ovis.visualization.visualizer import Visualizer
vis = Visualizer(view={
    "boundingbox_max" : [ 6.9281997680664063, 6.0, 30.0 ],
    "boundingbox_min" : [ -6.9281997680664063, -6.0, 0.0 ],
    "field_of_view" : 60.0,
    "front" : [ 0.51530507626740574, 0.67525725755210975, -0.52771992050352512 ],
    "lookat" : [ 0.0, 0.0, 15.0 ],
    "up" : [ -0.85631846790064325, 0.43037078955319003, -0.28548146180034029 ],
    "zoom" : 0.69999999999999996
})

mesh = o3d.io.read_triangle_mesh("C:/Users/TZMNL2/Downloads/NUT_JOB__Nut_Bolt_Washer_and_Threaded_Rod_Factory/files/bolt_25x8.stl")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
vis.draw_geometries([mesh])

In [3]:
pcd = mesh.sample_points_uniformly(number_of_points=50000)

plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0, 0, 1.0])
vis.draw_geometries([inlier_cloud, outlier_cloud])

Plane equation: 0.00x + -0.00y + 1.00z + -5.00 = 0


In [10]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: 0.00x + 0.00y + 1.00z + -0.00 = 0


In [5]:
from transformation_matrix import TransformationMatrix
import numpy as np
import copy

class Solution:
    def __init__(self, source):
        self.source = source

    @staticmethod
    def estimate_transformation(source, target):
        def fit_plane(pcd):
            plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
            inlier_cloud = pcd.select_by_index(inliers)
            return plane_model, inlier_cloud

        def get_bolt_head(pcd):
            plane_model, inlier_cloud = fit_plane(pcd)
            normal = plane_model[:3]
            center = np.median(inlier_cloud, axis=0)
            return normal, center

        def rotation():
            vec = np.cross(src_normal-tgt_normal)
            unit = vec / np.linalg.norm(vec)
            rotvec = unit * np.dot(src_normal-tgt_normal)
            return R.from_rotvec(rotvec).as_matrix()

        src_normal, src_center = get_bolt_head(source)
        tgt_normal, tgt_center = get_bolt_head(target)

        translation = tgt_center - src_center
        return TransformationMatrix.compose(rotation(), translation)

    @staticmethod
    def rmseT(g_est, g_gt):
        """
        Evaluation metrics
        - rmseP: Root square mean error of projection (rmseP) is
        calculated as the mean of point-point projection error after
        applying the transformation.
        - rmseT: Root square mean error of transformation (rmseT) represents
        the root-mean-square error between estimated transformation g_{est}
        and ground truth transformation g_{gt}.
        - RE: The rotation error (RE) is calculated
        as the Euclidean distance of rotation parameters between
        estimated r_{est} and ground truth r_{gt}.
        The rotation parameters are angles on three axes.
        - TE: The translation error (TE) is
        calculated as the Euclidean distance of translation parameters
        between estimated t_{est} and ground truth t_{gt}.
        """
        return np.sqrt(((g_est - g_gt) ** 2).mean())

    def evaluate_random():
        transform_truth = TransformationMatrix.make_random()
        transformed = copy.deepcopy(Solution.target).transform(transform_truth)
        transform_estimate = Solution.estimate_transformation(transformed)
        return Solution.rmseT(transform_truth, transform_estimate)

    def evaluate_batch(self, count=100):
        return np.mean([self.evaluate_random() for _ in range(count)])